# amazon-sagemaker-inference-using-tenstorhub-ready-model

This notebook is for building and deploying the code-sample as a BYOC (Bring your own container) as an Amazon SageMaker endpoint for object detection.

The model is a ready model downloaded directly from [Tensor Hub](https://www.tensorflow.org/hub/)

To use this notebook, you need to run it in AWS, preferably in Amazon SageMaker, with permissions to Amazon Elastic Container Registry, and Amazon Sagemaker

## Prerequisite

In [ ]:
# ECR Repository name
SVC='amazon-sagemaker-inference-using-tenstorhub-ready-model'
TAG='1.0'

In [ ]:
import boto3

sts = boto3.client("sts")
account_id = sts.get_caller_identity()["Account"]
aws_region = boto3.session.Session().region_name
ecr_url = "{}.dkr.ecr.{}.amazonaws.com".format(account_id, aws_region)

In [ ]:
# Create ECR repository for our docker image
!aws ecr --region $aws_region create-repository --repository-name $SVC > /dev/null

## Build

In [ ]:
# Let's build the docker image

full_reposotory_url = "{}/{}:{}".format(ecr_url, SVC, TAG)
!docker build -t $full_reposotory_url .

In [ ]:
# Login to ECR
!aws ecr get-login-password --region $aws_region | docker login --username AWS --password-stdin $ecr_url

In [ ]:
# Push docker image to the ECR repository

!docker push $full_reposotory_url

## Deploy

After we finished building the container image, we can start working on Amazon SageMaker.
We will need to follow couple of steps:

1. Creating a model that points to the container image
2. Creating a model endpoint configuration that contains the model, instance type etc...
3. Deploy - creating an endpoint from the model and endpoint configuraiton


In [ ]:
# First we will use the create_model to add the container image as an Amazon SageMaker model

from sagemaker import get_execution_role

role = get_execution_role()

client = boto3.client('sagemaker')

response = client.create_model(
    ModelName=SVC,
    PrimaryContainer={
        'ContainerHostname': SVC,
        'Image': full_reposotory_url,
        'ImageConfig': {
            'RepositoryAccessMode': 'Platform',
            'RepositoryAuthConfig': {
                'RepositoryCredentialsProviderArn': role
            }
        }
    },
    ExecutionRoleArn=role,
    EnableNetworkIsolation=False
)
print(response)

In [ ]:
# Now Let's configure an Endpoint configuration for the model we created

conf = client.create_endpoint_config(
    EndpointConfigName=SVC,
    ProductionVariants=[{
        'VariantName': 'default',
        'ModelName': SVC,
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'InstanceType': 'ml.g4dn.xlarge'
    }]
)

print(conf)

In [ ]:
# Now we deploy the model as an Amazon SageMaker endpoint

endpoint = client.create_endpoint(
    EndpointName=SVC,
    EndpointConfigName=SVC
)
print(endpoint)

In [ ]:
# Wait until the endpoint is InService
import time

endpoint_info = client.describe_endpoint(EndpointName=SVC)
endpoint_status = endpoint_info["EndpointStatus"]

while endpoint_status == "Creating":
    endpoint_info = client.describe_endpoint(EndpointName=SVC)
    endpoint_status = endpoint_info["EndpointStatus"]
    print("Endpoint status:", endpoint_status)
    if endpoint_status == "Creating":
        time.sleep(60)

print("Endpoint is in {}".format(endpoint_status))

## Testing the endpoint

The endpoint was built to download an images from Amazon S3 as an input, so let's create a new S3 bucket and upload the `./sample.jpg` file to the bucket and then we will invoke the endpoint with the appropriate information.

In [ ]:
# let's create a new bucket

# unique s3 bucket name
import uuid
bucket_name="sample-bucket-{}".format(str(uuid.uuid4())[:13])
print(bucket_name)
s3 = boto3.client('s3')


s3.create_bucket(Bucket=bucket_name,CreateBucketConfiguration={'LocationConstraint': aws_region})

In [ ]:
# Upload the sample.jpg file to the sample bucket
!aws s3 cp ./sample.jpg s3://$bucket_name/test/

In [ ]:
# Now we can test the SageMaker endpoint
# First invocation will be longer than all the next ones

import json

runtime = boto3.client('sagemaker-runtime')

payload = {
    "s3_bucket": bucket_name,
    "key_prefix": 'test',
    "file_name": 'sample.jpg'
}

response = runtime.invoke_endpoint(
    EndpointName=SVC,
    Body=json.dumps(payload),
    ContentType="application/json"
)

model_prediction = json.loads(response["Body"].read())
print(model_prediction)

In [ ]:
# Let's see the result

!pip install pillow
from PIL import Image, ImageDraw, ImageColor, ImageFont
import numpy as np

conf_score = 0.5

def draw_bounding_box_on_image(image, ymin, xmin, ymax, xmax, color, font, thickness=1, display_str_list=()):
    draw = ImageDraw.Draw(image)
    im_width, im_height = image.size
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                  ymin * im_height, ymax * im_height)
    draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
              (left, top)],
              width=thickness,
              fill=color)

    # If the total height of the display strings added to the top of the bounding
    # box exceeds the top of the image, stack the strings below the bounding box
    # instead of above.
    display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
    # Each display_str has a top and bottom margin of 0.05x.
    total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

    if top > total_display_str_height:
        text_bottom = top
    else:
        text_bottom = top + total_display_str_height
    # Reverse list and print from bottom to top.
    for display_str in display_str_list[::-1]:
        text_width, text_height = font.getsize(display_str)
        margin = np.ceil(0.05 * text_height)
        draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                        (left + text_width, text_bottom)],
                       fill=color)
        draw.text((left + margin, text_bottom - text_height - margin),
                  display_str,
                  fill='red',
                  font=font)
        text_bottom -= text_height - 2 * margin


def draw_bounding_box(image_path, inferred_image_result):
    colors = list(ImageColor.colormap.values())
    font = ImageFont.load_default()
    
    with Image.open(image_path) as img:
        for i in range(0, len(inferred_image_result)):
            if float(inferred_image_result[i]['confidence']) >= conf_score:
                ymin = float(inferred_image_result[i]['ymin'])
                xmin = float(inferred_image_result[i]['xmin'])
                ymax = float(inferred_image_result[i]['ymax'])
                xmax = float(inferred_image_result[i]['xmax'])
                obj_class = inferred_image_result[i]["class"]
                obj_confidence = float(inferred_image_result[i]["confidence"])

                # set bounding box display string
                display_str = "{}: {}%".format(obj_class, obj_confidence * 100)
                
                draw_bounding_box_on_image(img, ymin, xmin, ymax, xmax, colors[i], font, display_str_list=[display_str])

    img.save('./sample-boxed.jpg')

In [ ]:
# Draw our detection bounding boxes on our sample image
draw_bounding_box('./sample.jpg', model_prediction)


In [ ]:
# Lets see the image

from IPython.core.display import HTML

HTML('<img src="sample-boxed.jpg" alt="sample-boxed"'
     '<figcaption>sample-boxed.jpg</figcaption>The image has been downloaded from '
     'https://commons.wikimedia.org/wiki/File:Naxos_Taverna.jpg, '
     '<a href="https://en.wikipedia.org/wiki/GNU_Free_Documentation_License">License</a>')

### Cleanup

In [ ]:
# Amazon SageMaker

client.delete_endpoint(EndpointName=SVC)
client.delete_endpoint_config(EndpointConfigName=SVC)
client.delete_model(ModelName=SVC)


In [ ]:
# AWS S3

!aws s3 rm --recursive s3://$bucket_name
!aws s3 rb s3://mybucket

In [ ]:
# ECR
!aws ecr delete-repository --region $aws_region --repository-name $SVC --force